In [1]:
%%writefile filter.py
import pandas as pd
import os
import psycopg2
from configparser import ConfigParser
from sqlalchemy import create_engine
datafilePath=os.path.abspath('patients.csv')
if (os.path.isfile(datafilePath)):
    df=pd.read_csv(datafilePath)
    df=df[df.age.notna()]
    df.disease=df.disease.apply(lambda x: x.lower())
    df=df[((df.age>40)&(df.disease.str.contains('cancer')))]
    
    databaseIniFile=os.path.abspath('database.ini')
    if os.path.isfile('database.ini'):
        parser=ConfigParser()
        section='postgresql'
        parser.read(databaseIniFile)
        db={}
        if parser.has_section(section):
            for item in parser.items(section):
                db[item[0]]=item[1]
        conn=psycopg2.connect(**db)
        postgres_url = "postgresql://{user}:{password}@{host}:5432/{database}".format(**db)
        engine=create_engine(postgres_url)
        df.to_sql('patients',engine,if_exists='append')
    else:
        print("Database Initializer File Doen't Exist")
else:
    print("Record File Doesn't Exist")

Writing filter.py
